<a href="https://colab.research.google.com/github/coldbilberry/repo-gui/blob/main/%D0%A3%D1%80%D0%BE%D0%BA_16_%D0%A1%D0%BE%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D1%87%D0%B0%D1%82_%D0%B1%D0%BE%D1%82%D0%B0_%D0%B2_Telegram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install pyTelegramBotAPI
!pip install pygismeteo
!pip install geopy
!pip install tzwhere
!pip install pytz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.2/232.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 37.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tzwhere: filename=tzwhere-3.0.3-py3-none-any.whl size=23742581 sha256=ec60a39636c4ca2e48d42809dc45fd1638729ddc48399709cb49215d2072bcd5
  Stored in directory: /root/.cache/pip/wheels/fc/41/05/71acea338ac02030bc7d7cae140be7b033faaafab96a6dfdc2
Successfully built tzwhere


In [ ]:
import telebot
import requests
from config import TOKEN_API
from worker import get_weather, get_time
from telebot import types
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForCausalLM, AutoModelForSeq2SeqLM, pipeline

In [ ]:
#tokenizer = AutoTokenizer.from_pretrained("Den4ikAI/rubert_large_squad_2")
#model = AutoModelForCausalLM.from_pretrained("Den4ikAI/rubert_large_squad_2")

###############################

#tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/nllb-200-distilled-600M-ru-persona-chat")
#model = AutoModelWithLMHead.from_pretrained("DeepPavlov/nllb-200-distilled-600M-ru-persona-chat")
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/mbart-large-50-ru-persona-chat")
model = AutoModelForSeq2SeqLM.from_pretrained("DeepPavlov/mbart-large-50-ru-persona-chat")
###############################
token = TOKEN_API
bot = telebot.TeleBot(token)
text_infile = open('my_file.txt', 'r', encoding='utf-8').read()
qa_pipeline = pipeline(
     "question-answering",
     model="Den4ikAI/rubert_large_squad_2",
     tokenizer="Den4ikAI/rubert_large_squad_2")
@bot.message_handler(commands=['start'])
def start_message(message):
    bot.send_message(message.chat.id, 'Привет, я бот!\nЯ умею разговаривать, сообщать погоду и время. Также могу немного разбираюсь в медецине. \nНапиши свой запрос в сообщении ниже. \n Погода или время или еще какую байду. \n Если хочешь поговорить про медициу, то напиши запрос с пометкой "med"')
###############################
@bot.message_handler(content_types=['text'])
def repeat_all_messages(message): # Название функции не играет никакой роли
    if "med" in message.text.lower() or " med" in message.text.lower():
        msg = str(message.text.lower()).replace('med', '')
        predictions = qa_pipeline({
            'context': text_infile,
            'question': msg
        })
        bot.send_message(message.chat.id, predictions['answer'])
    elif "погод" in message.text.lower() or " погод" in message.text.lower():
        bot.send_message(message.chat.id, 'Введи название своего города и я скажу какая погода')
        bot.register_next_step_handler(message, weather_reply)
    elif "врем" in message.text.lower() or " врем" in message.text.lower():
        bot.send_message(message.chat.id, 'Введи название своего города и я скажу время')
        bot.register_next_step_handler(message, time_reply)
    else:
        prompt = message.text
        tokenized_prompt = tokenizer.encode(prompt, return_tensors="pt")
        generated_response = model.generate(tokenized_prompt, max_length=50)
        generated_response_text = tokenizer.decode(generated_response[0], skip_special_tokens=True)
        bot.send_message(message.chat.id, str(generated_response_text))
###############################
def weather_reply(message):
    city = message.text
    bot.send_message(message.chat.id, str(get_weather(city)))

def time_reply(message):
    city = message.text
    bot.send_message(message.chat.id, str(get_time(city)))
###############################
if __name__ == '__main__':
     bot.infinity_polling()